In [29]:
import gensim, logging
from model import BibleDatabase, BibleIterMode
from gensim.models import Word2Vec
import helper
import corpora

time: 2.67 ms


In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 1.65 ms


In [18]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
#%load_ext autotime
%reload_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime


[autoreload of autotime failed: Traceback (most recent call last):
  File "/home/npatta01/.virtualenvs/pyspark/local/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named autotime
]


In [19]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

time: 1.34 ms


[autoreload of autotime failed: Traceback (most recent call last):
  File "/home/npatta01/.virtualenvs/pyspark/local/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named autotime
]


# Load Corpora

In [64]:
db_path='../data/bible.db'

time: 1.23 ms


In [171]:
options = {
    "stem":True
    ,"stop_words":"../data/stopwords.txt"
    ,"filter_parts_of_speech":True
    ,"min_word_length":2
    ,"lower_case":False
    
}
ag_preprocesser = helper.Normalizer(**options)


time: 2.82 ms


In [173]:
def process(word):
    return ag_preprocesser.stemmer.lemmatize(word)

time: 1.42 ms


In [177]:
bible_verse_iter_mode = corpora.BibleSqliteCorpus(db_path,normalizer=ag_preprocesser,iter_mode=BibleIterMode.VERSE)

--- Completed 31104 BibleIterMode.VERSEs 
--- Completed 31104 BibleIterMode.VERSEs 
--- Completed 31104 BibleIterMode.VERSEs 
time: 1min 21s


In [178]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality
min_word_count = 40   # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words



time: 2.37 ms


In [179]:
?Word2Vec

time: 92.1 ms


In [180]:
context=5
min_word_count=5
downsampling=0

time: 120 ms


In [181]:
# Initialize and train the model (this will take some time)
print "Training Word2Vec model..."
model = Word2Vec(bible_verse_iter_mode, workers=num_workers, 
            size=num_features, min_count = min_word_count, 
            window = context, sample = downsampling, seed=1)

Training Word2Vec model...
--- Completed 31104 BibleIterMode.VERSEs 
--- Completed 31104 BibleIterMode.VERSEs 
time: 57.8 s


In [182]:
model.most_similar('god')

[(u'idol', 0.6522075533866882),
 (u'profaned', 0.5882677435874939),
 (u'derision', 0.5729453563690186),
 (u'molten', 0.5728124380111694),
 (u'provoke', 0.5676524639129639),
 (u'serve', 0.5506072640419006),
 (u'graven', 0.5498453378677368),
 (u'`Let', 0.544140636920929),
 (u'joyful', 0.5404459834098816),
 (u'jealousy', 0.5330997109413147)]

time: 12.1 ms


In [183]:
model.most_similar(process('God'))

[(u'faith', 0.769241988658905),
 (u'mystery', 0.7529106736183167),
 (u'Father', 0.7328070402145386),
 (u'gospel', 0.7173313498497009),
 (u'also', 0.7099764347076416),
 (u'grace', 0.7050580382347107),
 (u'may', 0.7037762999534607),
 (u'circumcision', 0.7007173299789429),
 (u'revelation', 0.6992039084434509),
 (u'suffering', 0.6925113201141357)]

time: 207 ms


In [184]:
model.most_similar(process('Mary'))

[(u'Simon', 0.7875022292137146),
 (u'Lazarus', 0.7857794165611267),
 (u'John', 0.7599195837974548),
 (u'James', 0.7563775181770325),
 (u'Philip', 0.7238887548446655),
 (u'centurion', 0.7212957739830017),
 (u'Cornelius', 0.7153438925743103),
 (u'Cephas', 0.7098357081413269),
 (u'Judas', 0.7069193124771118),
 (u'Cyrene', 0.7026147246360779)]

time: 87 ms


In [185]:
model.most_similar(process('Jesus'))

[(u'knowing', 0.873617947101593),
 (u'publican', 0.8631646037101746),
 (u'Peter', 0.8590988516807556),
 (u'Galilaeans', 0.8433119654655457),
 (u'answering', 0.8419257402420044),
 (u'Master', 0.8401651382446289),
 (u'John', 0.8359725475311279),
 (u'Barabbas', 0.8353924751281738),
 (u'Philip', 0.8310055136680603),
 (u'Simon', 0.8285192251205444)]

time: 82.7 ms


In [187]:
model.most_similar(positive=['crucified'],negative=['dead'], topn=30)

[(u'sinful', 0.5149369239807129),
 (u'withdrew', 0.5047765970230103),
 (u'God', 0.46968695521354675),
 (u'hour', 0.46700260043144226),
 (u'teaching', 0.45644107460975647),
 (u'immediately', 0.4493235945701599),
 (u'Nazarene', 0.44469577074050903),
 (u'Barabbas', 0.4441291391849518),
 (u'perceiving', 0.43706417083740234),
 (u'everywhere', 0.4348283112049103),
 (u'Nazareth', 0.4287862181663513),
 (u'gospel', 0.4257271885871887),
 (u'word', 0.42475029826164246),
 (u'upon', 0.4238258898258209),
 (u'Lord', 0.41975855827331543),
 (u'Capernaum', 0.4139862358570099),
 (u'Christ', 0.41335874795913696),
 (u'diadem', 0.4118346571922302),
 (u'Zeboiim', 0.410378634929657),
 (u'Andrew', 0.40710189938545227),
 (u'manifested', 0.40560412406921387),
 (u'ministration', 0.4044244885444641),
 (u'Philippi', 0.4043862521648407),
 (u'Baptist', 0.40156278014183044),
 (u'foreordained', 0.3963351249694824),
 (u'`places', 0.39442431926727295),
 (u'day', 0.3938901722431183),
 (u'baptizing', 0.39247822761535645),


time: 176 ms


In [188]:
model.most_similar(positive=['woman', 'king'], negative=['man'],topn=30)

[(u'eunuch', 0.6231749057769775),
 (u'Babylon', 0.5704138875007629),
 (u'army', 0.5494177937507629),
 (u'captive', 0.5361217260360718),
 (u'noble', 0.5318084955215454),
 (u'Chaldeans', 0.5216084122657776),
 (u'Riblah', 0.5147387385368347),
 (u'carried', 0.5135143995285034),
 (u'Nebuchadrezzar', 0.5079478621482849),
 (u'Tyre', 0.5046406984329224),
 (u'captivity', 0.503948986530304),
 (u'enchanter', 0.495053231716156),
 (u'Judah', 0.4884354770183563),
 (u'war', 0.4844403862953186),
 (u'found', 0.4826550781726837),
 (u'Ahab', 0.47736504673957825),
 (u'province', 0.47612956166267395),
 (u'queen', 0.47476378083229065),
 (u'gathered', 0.47269436717033386),
 (u'Jews', 0.47055792808532715),
 (u'ruler', 0.46977996826171875),
 (u'accusation', 0.46865618228912354),
 (u'Nineveh', 0.468523770570755),
 (u'smote', 0.46500006318092346),
 (u'join', 0.46498116850852966),
 (u'Hormah', 0.46471381187438965),
 (u'Ethiopians', 0.4641486704349518),
 (u'prince', 0.4615289568901062),
 (u'Nebuchadnezzar', 0.4587

time: 84.2 ms


In [189]:
?model.most_similar

time: 85 ms


In [190]:
model.doesnt_match("Abraham Jesus John".split())

'Abraham'

time: 82.4 ms


In [191]:
v= model.vocab.keys()

time: 122 ms


In [192]:
len(v)

5196

time: 151 ms


In [193]:
[w for w in v if 'people' in w]

[u'people']

time: 123 ms


In [194]:
model.most_similar(positive=['servant','king'],negative=[], topn=30)

[(u'sent', 0.6415464878082275),
 (u'prophet', 0.6002540588378906),
 (u'answered', 0.5972616672515869),
 (u'Daniel', 0.5824201703071594),
 (u'spake', 0.5782758593559265),
 (u'Jews', 0.5712401270866394),
 (u'Haman', 0.5675198435783386),
 (u'Esther', 0.5670063495635986),
 (u'eunuch', 0.5656139254570007),
 (u'showing', 0.5633755922317505),
 (u'condemned', 0.5597923994064331),
 (u'openly', 0.5468555092811584),
 (u'prayer', 0.5420787930488586),
 (u'found', 0.5418552160263062),
 (u'ruler', 0.5415225028991699),
 (u'Belteshazzar', 0.5319534540176392),
 (u'Mordecai', 0.5310443043708801),
 (u'Pharisees', 0.5263097286224365),
 (u'demon', 0.5245246291160583),
 (u'saying', 0.5153595805168152),
 (u'heard', 0.5080572962760925),
 (u'Satan', 0.5069719552993774),
 (u'also', 0.5013088583946228),
 (u'hypocrite', 0.5008547306060791),
 (u'lord', 0.5004507899284363),
 (u'Pilate', 0.5001140832901001),
 (u'sorcerer', 0.49982306361198425),
 (u'preach', 0.49521833658218384),
 (u'persuade', 0.49339866638183594),
 

time: 86.8 ms


In [195]:
model.similarity('Jesus','cow')

-0.4645863476104557

time: 80.2 ms


In [196]:
model.most_similar('praying')

[(u'angel', 0.5989052653312683),
 (u'worshipped', 0.5422489047050476),
 (u'speaking', 0.5366223454475403),
 (u'standing', 0.5217623114585876),
 (u'swiftly', 0.5031404495239258),
 (u'contrary', 0.5027492046356201),
 (u'declaring', 0.4984506070613861),
 (u'odor', 0.49032509326934814),
 (u'Elijah', 0.48931944370269775),
 (u'appeared', 0.48928216099739075)]

time: 87.9 ms


In [200]:
process('speak')

'speak'

time: 2.33 ms


In [206]:
model.most_similar(positive=['disciple','mother'],negative=['female'], topn=30)

[(u'Cornelius', 0.810705304145813),
 (u'Mary', 0.8014918565750122),
 (u'Philip', 0.7885518670082092),
 (u'Peter', 0.7860925793647766),
 (u'Paul', 0.774225115776062),
 (u'John', 0.7728469967842102),
 (u'Master', 0.7718327641487122),
 (u'centurion', 0.7713876962661743),
 (u'apostle', 0.7698001861572266),
 (u'Simon', 0.7626448273658752),
 (u'Nazareth', 0.7578897476196289),
 (u'parent', 0.7542701959609985),
 (u'boldly', 0.7539756298065186),
 (u'judgment-seat', 0.7531784772872925),
 (u'Martha', 0.7522764205932617),
 (u'therefore', 0.7469778656959534),
 (u'Pilate', 0.7396385669708252),
 (u'Barabbas', 0.7394375205039978),
 (u'Timothy', 0.738165020942688),
 (u'amazed', 0.7355075478553772),
 (u'called', 0.7348423600196838),
 (u'answering', 0.7348194718360901),
 (u'marvelled', 0.7264348268508911),
 (u'Brethren', 0.7227752804756165),
 (u'Ananias', 0.722525417804718),
 (u'Woman', 0.72185218334198),
 (u'Galatia', 0.7202388048171997),
 (u'Rabbi', 0.7144089937210083),
 (u'Lazarus', 0.7142627239227295

time: 7.46 ms


In [197]:
model.most_similar(positive=['Heaven'],negative=['happy'], topn=30)

[(u'stumbling', 0.6919000148773193),
 (u'harp', 0.6115476489067078),
 (u'`the', 0.5961785316467285),
 (u'hypocrite', 0.5656452775001526),
 (u'stork', 0.5573034286499023),
 (u'idle', 0.5569040775299072),
 (u'deceiver', 0.5525301694869995),
 (u'heaven', 0.5368068814277649),
 (u'court', 0.5286533832550049),
 (u'self', 0.528278112411499),
 (u'door', 0.5247617363929749),
 (u'graven', 0.5235903263092041),
 (u'teaching', 0.5225210785865784),
 (u'round', 0.5186126232147217),
 (u'sound', 0.5173542499542236),
 (u'wizard', 0.5169318914413452),
 (u'ascend', 0.5158020853996277),
 (u'Behold', 0.513918936252594),
 (u'endured', 0.5134844183921814),
 (u'place', 0.5128961205482483),
 (u'temple', 0.5121800899505615),
 (u'synagogue', 0.5118653774261475),
 (u'`of', 0.511333703994751),
 (u'ministration', 0.5112622380256653),
 (u'carpenter', 0.504682183265686),
 (u'scripture', 0.5013973116874695),
 (u'sounding', 0.50049889087677),
 (u'earthquake', 0.4904426038265228),
 (u'Greeks', 0.48979565501213074),
 (u'c

time: 82.3 ms
